<a href="https://colab.research.google.com/github/iPrinka/MITx-Micromasters-Statistics-Data-Science/blob/main/one_league_w5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Classifying Handwritten Digits

**OBJECTIVES**

- Use SVM to classifiy handwritten digits
- Explore different kernels in SVM
- Use `sklearn` to fine tune estimators performance
- Use `Pipeline` and `GridSearchCV` in sklearn to optimize performance

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.linear_model import LogisticRegression, Perceptron, LinearRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA 
from sklearn.metrics import accuracy_score

In [ ]:
from keras.datasets import mnist

In [ ]:
#load the data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
#visualize the first datapoint
plt.imshow(x_train[0], cmap = 'gray');

In [ ]:
#examine unique values in y


#### Preparing the Data

- Flatten the images
- Scale the data


In [ ]:
#examine the shape


In [ ]:
#reshape x_train


In [ ]:
#reshape x_test


**StandardScaler**

$$x_i = \frac{x_i - \mu}{\sigma}$$

In [ ]:
#by hand - find mean and standard deviation


In [ ]:
#apply them


In [ ]:
#using sklearn -- instantiate


In [ ]:
#fit


In [ ]:
#transform


In [ ]:
#all in one with fit_transform


#### `LinearRegression`

In [ ]:
#instantiate


In [ ]:
#fit


In [ ]:
#predict


In [ ]:
#score it


#### `LinearSVC`

In [ ]:
#instantiate


In [ ]:
#fit


In [ ]:
#score


#### `LogisticRegression`

How to implement a multinomial model?  Inspect the documentation [here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) and implement `LogisticRegression` as a multi-class model using multinomial regression.  

In [ ]:
#instantiate


In [ ]:
#fit


In [ ]:
#score on train and test


### Dimensionality Reduction with `PCA`

Important notes:

- Data should be scaled
- Transformer with `fit` and `transform` methods

[docs](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)

In [ ]:
#instantiate


In [ ]:
#fit


In [ ]:
#transform


In [ ]:
#shape of results?


In [ ]:
#redo with 10 principal components -- instantiate


In [ ]:
#transform


In [ ]:
#visualize first two components with classes


#### SVM with Polynomial kernel

Examine the documentation for the `SVC` classifier [here](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).  Implement a polynomial kernel with degree 3 on the PCA data from above and score this on the test data.  Compare this to an SVC with a gaussian (`rbf`) kernel.  Which performed. better?

In [ ]:
#instantiate model with polynomial degree 3


In [ ]:
#fit on training data


In [ ]:
#score on test data


In [ ]:
#instantitate other model with rbf kernel


In [ ]:
#fit 


In [ ]:
#score


#### Streamlining the Modeling Process

- `Pipeline` to combine transformations and preprocessing steps
- `GridSearchCV` to search different parameter combinations

In [ ]:
#Pipeline to scale, pca, svc


In [ ]:
#parameter grid to search 


In [ ]:
#grid search with pipeline


#### Practice: FashionMNIST

Below, a dataset of images of fashion items is loaded and displayed.  Your goal is to build a Support Vector Machine classifier considering different kernels and preprocessing variations.

In [ ]:
from keras.datasets import fashion_mnist

In [ ]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [ ]:
fig, ax = plt.subplots(1, 5, figsize = (25, 5))
for i in range(5):
  ax[i].imshow(x_train[i], cmap = 'gray')

In [ ]:
x_train.shape